In [8]:
from qiskit import IBMQ, transpile, assemble, schedule
from qiskit.circuit import Parameter, QuantumCircuit
#from qiskit.quantum_info import Pauli

from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.mappers.second_quantization import FermionicMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

from qiskit.opflow import (I, X, Y, Z, Zero, One, MatrixEvolution, PauliTrotterEvolution, Suzuki,
                           StateFn, Zero, One, PauliExpectation, PauliOp, SummedOp, OperatorBase)

import numpy as np
import scipy.linalg as lng
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [5, 5]

In [54]:
htst = FermionicOp('+IIIIIIII-')
htst = FermionicOp('IIIIINIIII')
htst = FermionicOp('IIIIIII+-I')

In [55]:
mapper = JordanWignerMapper()
mapper = BravyiKitaevMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

ht_pauli = converter.convert(htst)

print(ht_pauli)


0.25 * XXXIIIIIII
+ -0.25j * XYXIIIIIII
+ 0.25j * XXYZZIZIII
+ 0.25 * XYYZZIZIII


In [138]:
def parity_set(j, n):
    """
    Computes the parity set of the j-th orbital in n modes.

    Args:
        j (int) : the orbital index
        n (int) : the total number of modes

    Returns:
        numpy.ndarray: Array of mode indices
    """
    
    indices = np.array([])
    if n % 2 != 0:
        return indices

    if j < n / 2:
        indices = np.append(indices, parity_set(j, n / 2))
        #print("j =", j, ", n =", n, ", set = ", indices)
    else:
        indices = np.append(indices, np.append(
            parity_set(j - n / 2, n / 2) + n / 2, n / 2 - 1))
        #print("j =", j, ", n =", n, ", set = ", indices)
    return indices

def update_set(j, n):
    """
    Computes the update set of the j-th orbital in n modes.

    Args:
        j (int) : the orbital index
        n (int) : the total number of modes

    Returns:
        numpy.ndarray: Array of mode indices
    """
    indices = np.array([])
    if n % 2 != 0:
        return indices
    if j < n / 2:
        indices = np.append(indices, np.append(
            n - 1, update_set(j, n / 2)))
        #print("j =", j, ", n =", n, ", set = ", indices)
    else:
        indices = np.append(indices, update_set(j - n / 2, n / 2) + n / 2)
        #print("j =", j, ", n =", n, ", set = ", indices)
    return indices

def flip_set(j, n):
    """
    Computes the flip set of the j-th orbital in n modes.

    Args:
        j (int) : the orbital index
        n (int) : the total number of modes

    Returns:
        numpy.ndarray: Array of mode indices
    """
    indices = np.array([])
    if n % 2 != 0:
        return indices
    if j < n / 2:
        indices = np.append(indices, flip_set(j, n / 2))
        #print("j =", j, ", n =", n, ", set = ", indices)
    elif n / 2 <= j < n - 1:
        indices = np.append(indices, flip_set(j - n / 2, n / 2) + n / 2)
        #print("j =", j, ", n =", n, ", set = ", indices)
    else:
        indices = np.append(np.append(indices, flip_set(
            j - n / 2, n / 2) + n / 2), n / 2 - 1)
        #print("j =", j, ", n =", n, ", set = ", indices)
    return indices


In [171]:
flip_set(5,16)

array([4.])

In [172]:
update_set(5,16)

array([15.,  7.])

In [170]:
parity_set(5,16)

array([4., 3.])

In [173]:
htst = FermionicOp('IIIII+IIIIIIIIII')

mapper = BravyiKitaevMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

ht_pauli = converter.convert(htst)

print(ht_pauli)


0.5j * XIIIIIIIXIYIZIII
+ 0.5 * XIIIIIIIXIXZZIII


In [183]:
def gx_BK(i,n):
    U = update_set(i,n)
    P = parity_set(i,n)
    F = flip_set(i,n)
    label = ['I' for j in range(n)]
    label[i] = 'X'
    for j in U:
        label[int(j)] = 'X'
    for j in P:
        label[int(j)] = 'Z'
    label = np.flip(label)
    out = ''
    for l in label:
        out = out +l
    return out

def gy_BK(i,n):
    U = update_set(i,n)
    P = parity_set(i,n)
    F = flip_set(i,n)
    label = ['I' for j in range(n)]
    label[i] = 'Y'
    for j in U:
            label[int(j)] = 'X'
    if i%2 == 0:
        for j in P:
            label[int(j)] = 'Z'
    else:
        for j in P:
            if j not in F:
                label[int(j)] = 'Z'
    label = np.flip(label)
    out = ''
    for l in label:
        out = out +l
    return out
               

'XIIIIIIIXIIIXXZI'

In [249]:
print(gx_BK(5,16))
print(gy_BK(5,16))

XIIIIIIIXIXZZIII
XIIIIIIIXIYIZIII


In [250]:
htst = FermionicOp('IIIII+IIIIIIIIII')

mapper = BravyiKitaevMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

ht_pauli = converter.convert(htst)

print(ht_pauli)

0.5j * XIIIIIIIXIYIZIII
+ 0.5 * XIIIIIIIXIXZZIII
